<left>
<table style="margin-top:0px; margin-left:0px;">
<tr>
  <td><img src="https://raw.githubusercontent.com/worm-portal/WORM-Figures/master/style/worm.png" alt="WORM" title="WORM" width=50/></td>
  <td><h1 style=font-size:30px>Activity Diagrams Part 4</h1><br />
</tr>
</table>
</left>

### What's covered in this demo?

 - Plotting as a function of charged chemical species
 - Reproducing [Figure 8A](https://ars.els-cdn.com/content/image/1-s2.0-S2590197421000070-gr8_lrg.jpg) in [Dick 2021](https://doi.org/10.1016/j.acags.2021.100059), a mineral predominance diagram depicting a Cu-Fe-S-O-H system balanced by O<sub>2</sub>
 - Making black and white figures
 - Adding mineral saturation lines with `add_saturation_lines`
 - Saving figures as files (e.g., PNG, JPG, WEBP, SVG, PDF, HTML)

### Introduction

In this demo, let's reproduce Figure A8 (left panel) in [Dick 2021](https://doi.org/10.1016/j.acags.2021.100059):

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S2590197421000070-gr8_lrg.jpg" alt="Fig8A in Dick 2021" title="Fig8A in Dick 2021" width=75%/>

Going through the step-by-step process of recreating this figure will touch on some new concepts and features of pyCHNOSZ that might be useful when creating activity diagrams for your own projects.

### Getting ready

Just like in Part 1 and 2 of the Activity Diagram demo, we will import the pyCHNOSZ Python package and the [WORM thermodynamic database](https://worm-portal.asu.edu/docs/database/).

In [ ]:
from pyCHNOSZ import *
_ = thermo("WORM")

First, let's load a set of basis species with the `basis` function. The goal here is to select basis species that can be used to "build" all of the minerals depicted in Fig8A.

Here is where you have the opportunity to choose basis species that are most relevant to your system. Would you like to use SO4-2 to provide the sulfur in the system? Or elemental sulfur? Or what about carbon? Should you use CO2, CH4, or something else? You get to decide how the model is set up based on your best judgement and what you are trying to model.

In our case, [Dick 2021](https://doi.org/10.1016/j.acags.2021.100059) provides code for CHNOSZ (in R) in their supplementary information that lets us follow along using pyCHNOSZ (in Python). They set up their basis set like this:

In [ ]:
basis(["Fe+2", "Cu+", "H2S(g)", "O2(g)", "H2O", "H+"])

The author also boosted the log activity of H2S(g) to 2. Let's do that too.

Note that when we defined our basis species in the cell above, we used "H2S(g)" to represent sulfur. Then in the dataframe output, the species was renamed "H2S" and is listed as a "gas" under the state column (same with "O2(g)" and "O2").

This is important to note because when changing the activity of a basis species, you have to use the name that appears in the dataframe output from `basis`. In this case, we want to change the log activity of H2S(g), so we use "H2S" because that is what's in the table given by `basis`:

In [ ]:
basis("H2S", 2)

In the table above, the logact of "H2S", our H2S(g), has been changed to 2.

Okay, now we are ready to define the minerals we would like to plot. Let's begin with the minerals with regions in Fig8A. The abbreviations used in the figure refer to the following minerals:

Py - pyrite

Mag - magnetite

Cv - covellite

Cpp - chalcopyrite

Bn - bornite

Let's put a list of these mineral names into the `species` function:

In [ ]:
species(["pyrite", "magnetite", "covellite", "chalcopyrite", "bornite"])

Now that we have our `basis` and `species` loaded, it's time for `affinity`. This is where we get to decide what variables to plot on our axes, the ranges of those variables, the temperature, and the pressure.

The annotation in Fig8A tells us that the temperature is 400 °C and 2000 bars. That's no trouble - we know from previous demos how to adjust temperature and pressure in the `affinity` function. But what about our x and y variables?

Fig8A has some complicated-looking x and y axes labels:

log(a<sub>Cu<sup>+</sup></sub>/a<sub>H<sup>+</sup></sub>)

log(a<sub>Fe<sup>+2</sup></sub>/a<sup>2</sup><sub>H<sup>+</sup></sub>)

They represent activities of Cu+ or Fe+2 ions expressed ratios of H+ raised to the power of the ion's charge. This is commonly done in geochemical predominance diagrams to reduce the number of variables necessary for plotting. In effect, the system's pH is effectively folded into the visualization.

Fortunately for us, we don't need to do anything fancy to take these ion ratios into account because pyCHNOSZ automatically does it for us. We'll soon see this when we create our first draft of the diagram later.

For now, the takeaway is that Cu+ and Fe+2 are going to be used as our x and y axis parameters, respectively, when we fill out our `affinity` function. Let's use a Python dictionary to specify our parameters in `affinity` because we have basis species with "+" or "-" in our axes (see Part 2 for mo):

In [ ]:
a = affinity(**{"Cu+":[-8, 2], "Fe+2":[-4, 12], "T":400, "P":2000})

Okay, let's plot the output of our `affinity` calculation using `diagram`.

The title of Fig8A specifies that the system is balanced by O<sub>2</sub>. We can specify that with the `balance` parameter:

In [ ]:
d = diagram(a, balance="O2", width=500, height=400, interactive=True)

Not a bad start! The mineral predominance regions are in the correct positions and have the right shapes. The axes labels need to be adjusted to specify that they are ratios of H+. We can do that with the ratio label function `ratlab`.

In [ ]:
d = diagram(a,
            balance="O2",
            xlab=ratlab("Cu+"), # x axis label
            ylab=ratlab("Fe+2"), # y axis label
            width=500, height=400, interactive=True)

Now that the axes are properly labeled, we could pack up and call it a day. But let's go further to replicate Fig8A as closely as we can. We can make the diagram black-and-white by specifying `fill`, `col`, and `lwd` and add a title with `main`:

In [ ]:
d = diagram(a,
            balance="O2",
            xlab=ratlab("Cu+"), # x axis label
            ylab=ratlab("Fe+2"), # y axis label
            fill="none", # no fill
            col="black", # black lines
            lwd=0.5, # line thickness
            main="Cu-Fe-S-O-H; 1° balance: O<sub>2</sub>", # plot title in HTML format
            width=500, height=400, interactive=True)

We're getting really close now. The only difference between our plot and Fig8A is the use of mineral abbreviations. Let's implement abbreviated mineral names in our plot, too.

Each entry in the WORM database has an index, a number representing the order in which it appears. This number is shown in the 'ispecies' column of `species()`:

In [ ]:
species()

We can grab a list of mineral indices from the 'ispecies' column with:

In [ ]:
species_indices = list(species()["ispecies"])
species_indices

Then we can use this list along with the `info` function to look up the thermodynamic data for these minerals:

In [ ]:
info(species_indices)

Notice that there is a column for chemical species abbreviations, called 'abbrv'. Much like before, we can grab a list of these abbreviations with:

In [ ]:
species_abbrv = list(info(species_indices)["abbrv"])
species_abbrv

Now we have a list of mineral abbreviations that will dynamically update upon re-running this notebook if we ever decide to change which minerals we include in our plot.

We can feed these abbreviations into our diagram using the `names` parameter:

In [ ]:
_ = diagram(a,
            balance="O2",
            xlab=ratlab("Cu+"), # x axis label
            ylab=ratlab("Fe+2"), # y axis label
            fill="none", # no fill
            col="black", # black lines
            lwd=0.5, # line thickness
            main="Cu-Fe-S-O-H; 1° balance: O<sub>2</sub>", # plot title in HTML format
            names=species_abbrv, # predominance region names
            width=500, height=400, interactive=True)

The original Fig8A has an annotation in the upper left corner that displays the temperature and pressure of the system. Let's add that to our plot too using the `annotation` and `annotation_coords` parameters.

In [ ]:
_ = diagram(a,
            balance="O2",
            xlab=ratlab("Cu+"), # x axis label
            ylab=ratlab("Fe+2"), # y axis label
            fill="none", # no fill
            col="black", # black lines
            lwd=0.5, # line thickness
            main="Cu-Fe-S-O-H; 1° balance: O<sub>2</sub>", # plot title in HTML format
            names=species_abbrv, # predominance region names
            annotation="400 °C, 2000 bar", # annotation text
            annotation_coords=[0, 1], # annotation coordinates
            width=500, height=400, interactive=True)

Annotation coordinates are given as a list of two numbers. For example, ours is:

`annotation_coords=[0, 1],`

The first number is the x position of the annotation, ranging from 0 (left) to 1 (right), with 0.5 in the middle. The second number is the y position, from 0 (bottom) to 1 (top). Since these coordinates are `[0, 1]`, the annotation appears in the left top corner.

The last thing we need to do to replicate Fig8A is to add the mineral saturation lines. For this, we will need to store our diagram inside of a Python variable. We can do this by setting `fig_out=True`, which makes the `diagram` function output the figure object itself instead of just displaying it.

When `fig_out=True`, the `diagram` function outputs two things at the same time. The first output is a dataframe containing some plot data. We don't care about that right now, so we'll just assign that output to a throwaway variable, `_`. The second output is the figure object. This is what we want, so let's assign it to the variable `d`. Note that `_` and `d` are separated by a comma, like this:

```
_, d = diagram(a,
        ...
```

In [ ]:
_, d = diagram(a,
            balance="O2",
            xlab=ratlab("Cu+"), # x axis label
            ylab=ratlab("Fe+2"), # y axis label
            fill="none", # no fill
            col="black", # black lines
            lwd=0.5, # line thickness
            main="Cu-Fe-S-O-H; 1° balance: O<sub>2</sub>", # plot title in HTML format
            names=species_abbrv, # predominance region names
            annotation="400 °C, 2000 bar", # annotation text
            annotation_coords=[0, 1], # annotation coordinates
            fig_out=True,
            width=500, height=400, interactive=True)

The figure is now stored in the variable `d`:

In [ ]:
d

The interactive plots you can make with pyCHNOSZ are examples of Plotly plots. If you have some Python know-how, you can customize a Plotly plot like this one in all sorts of ways (but that is beyond the scope of this tutorial).

For now, let's focus on adding saturation lines to our diagram. Fig8A has the following minerals appearing as saturation lines. Let's generate a new set of `species`. These will use the same `basis` we previously defined.

In [ ]:
species(["pyrrhotite", "ferrous-oxide", "chalcocite", "cuprite"])

Now let's re-run our `affinity` calculation for these new minerals. We need to make sure the parameters match those we used previously, since we will be plotting these saturation lines atop our other plot.

In [ ]:
a = affinity(**{"Cu+":[-8, 2], "Fe+2":[-4, 12], "T":400, "P":2000})

The last step is to use the `add_saturation_lines` function. The first parameter is the output of `affinity`, `a`. The second is our `diagram`, `d`

In [ ]:
d_sat = add_saturation_lines(a, d)
d_sat

The lines are black by default. Let's make them a nice blue color:

In [ ]:
d_sat = add_saturation_lines(a, d, line_color="cornflowerblue")
d_sat

Or we can exert more control over saturation line colors, styles, and width. Line types are cycled before moving onto the next color:

In [ ]:
d_sat = add_saturation_lines(a, d,
                             line_width=1.5,
                             line_type=["dot", "dashdot"],
                             line_color=["cornflowerblue", "orange"],
                             )
d_sat

Now that we have this beautiful plot, we can export it as a static image with `write_image`. A few different filetypes are available, including rasters (png, jpg, webp), and vectors (svg, pdf, eps). Let's export our image as a pdf file:

In [ ]:
# Wait a second before writing image to allow
# the diagram to render fully before writing.
# Prevents a 'loading' message from appearing in
# the corner of the resulting PDF.
import time
time.sleep(1) # waits 1 second

# write the image to a PDF
d_sat.write_image("Fig8A.pdf")

Check your WORM file browser. A file called `Fig8A.pdf` should have appeared (or will in a moment).

If you would like to download this figure to your computer, you can right-click (control+click on a Mac) on the file in the WORM file browser and select "Download".

It's also possible to export the interactive diagram itself as an HTML file that can be embedded in websites or viewed on the WORM Portal:

In [ ]:
d_sat.write_html("Fig8A.html")

If you open this HTML file in your WORM file browser, you might not be able to see it until you click the "Trust HTML" button in the upper left corner of the tab that appears. Once you are able to see the plot, try mousing over it to confirm that it is interactive.

Congratulations! You've created a slick-looking predominance diagram with pyCHNOSZ.

---

**Lingering questions:**
- Why don't some of the mineral saturation lines in our plot match perfectly with Fig 8A from Dick 2021? For example, the vertical saturation line for cuprite appears at log(*a*<sub>Cu<sup>+</sup></sub>/*a*<sub>H<sup>+</sup></sub>) = 1.35 in our plot, but it looks more like 1.75 in Fig 8A. This is because Dick 2021 uses the OBIGT ("One Big Table") thermodynamic database that primarily uses minerals from [Helgeson et al., 1978](http://www.dewcommunity.org/uploads/4/1/7/6/41765907/helgeson_et_al_1978.pdf) while also mixing in minerals from [Berman 1988](https://doi.org/10.1093/petrology/29.2.445) that have their own specialized set of equations of state. The WORM database used in this demo does not include minerals from Berman 1988 in order to maintain internally consistent mineral properties.
- If you can think of any questions that ought to be answered in this section, feel free to contact WORM administrator Grayson Boyer (gmboyer at asu dot edu).

End of demo